In [406]:
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToJson
import urllib.request
import time
import pytz
import json
import pandas as pd
from datetime import datetime
from datetime import date

def scheduleDateTime(stop_time):
    if pd.isna(stop_time):
        return 0
    time_obj = datetime.strptime(stop_time, "%H:%M:%S").time()
    dt_with_time = datetime.combine(dt, time_obj)
    dt_with_time = pytz.timezone('Europe/Warsaw').localize(dt_with_time)
    return dt_with_time

feed = gtfs_realtime_pb2.FeedMessage()
response = urllib.request.urlopen('https://mkuran.pl/gtfs/wkd.pb')
feed.ParseFromString(response.read())

rozklad = pd.read_csv(r'C:\Users\hmackowski\Desktop\python scripts\wkd\stop_times.txt', sep=',')
print(rozklad)
json_wkd = MessageToJson(feed)
#print(json_wkd)
json_wkd=json.loads(json_wkd)
json_wkd=json_wkd['entity']
#print(json_wkd)



     trip_id arrival_time departure_time stop_id  stop_sequence
0      C-303     05:05:00       05:05:00   wsrod              0
1      C-303     05:07:00       05:07:00   wocho              1
2      C-303     05:10:00       05:10:00   wzach              2
3      C-303     05:12:00       05:12:00   wreor              3
4      C-303     05:14:00       05:14:00   walje              4
...      ...          ...            ...     ...            ...
6283   D-101     25:12:00       25:12:00   brzoz             21
6284   D-101     25:14:00       25:14:00   gmokr             22
6285   D-101     25:16:00       25:16:00   gmpia             23
6286   D-101     25:18:00       25:18:00   gmjor             24
6287   D-101     25:23:00       25:23:00   gmrad             25

[6288 rows x 5 columns]


In [407]:
df = pd.json_normalize(json_wkd)


In [408]:
df = pd.json_normalize(json_wkd, record_path=['tripUpdate','stopTimeUpdate'],meta=['id'])
tz = pytz.timezone('Europe/Warsaw')
df['departure.time'] = pd.to_datetime(df['departure.time'], unit='s').apply(lambda x: pd.Timestamp(x).tz_localize(None).tz_localize('UTC').tz_convert(tz))
print(df)

    stopId            departure.time      id
0    gmokr 2023-03-28 23:14:00+02:00   D-151
1    kazim 2023-03-28 23:09:00+02:00   D-151
2    wrako 2023-03-28 22:31:00+02:00   D-151
3    gmjor 2023-03-28 23:18:00+02:00   D-151
4    brzoz 2023-03-28 23:12:00+02:00   D-151
..     ...                       ...     ...
244  milgr 2023-03-28 22:39:00+02:00  D-5337
245  poles 2023-03-28 21:42:00+02:00  D-3085
246  milgr 2023-03-28 21:44:00+02:00  D-3085
247  poles 2023-03-28 21:42:00+02:00  D-5335
248  milgr 2023-03-28 21:44:00+02:00  D-5335

[249 rows x 3 columns]


In [409]:
actual = df[df['id'] == 'D-3097']
  
print(actual)

rozklad_filtr = rozklad[rozklad['trip_id'] == 'D-3097']
  
print(rozklad_filtr)

    stopId            departure.time      id
88   plglo 2023-03-28 22:32:00+02:00  D-3093
89   wrako 2023-03-28 21:56:00+02:00  D-3093
90   malic 2023-03-28 22:07:00+02:00  D-3093
91   wsalo 2023-03-28 21:58:00+02:00  D-3093
92   poles 2023-03-28 22:37:00+02:00  D-3093
93   wocho 2023-03-28 21:47:00+02:00  D-3093
94   wsrod 2023-03-28 21:45:00+02:00  D-3093
95   milgr 2023-03-28 22:39:00+02:00  D-3093
96   prusz 2023-03-28 22:12:00+02:00  D-3093
97   regul 2023-03-28 22:05:00+02:00  D-3093
98   micha 2023-03-28 22:03:00+02:00  D-3093
99   walje 2023-03-28 21:54:00+02:00  D-3093
100  wzach 2023-03-28 21:50:00+02:00  D-3093
101  otreb 2023-03-28 22:21:00+02:00  D-3093
102  komor 2023-03-28 22:15:00+02:00  D-3093
103  wreor 2023-03-28 21:52:00+02:00  D-3093
104  plwsc 2023-03-28 22:23:00+02:00  D-3093
105  nwwar 2023-03-28 22:17:00+02:00  D-3093
106  opacz 2023-03-28 22:00:00+02:00  D-3093
107  kanie 2023-03-28 22:19:00+02:00  D-3093
108  twork 2023-03-28 22:10:00+02:00  D-3093
     trip_

In [413]:
# merge dataframes based on key_col1 and key_col2
merged_df = pd.merge(actual, rozklad, left_on=['id','stopId'], right_on=['trip_id','stop_id'], how='left')

merged_df = merged_df.drop(columns=['trip_id','stop_id','departure_time'])
merged_df = merged_df.sort_values(by='stop_sequence')

merged_df['arrival_time'] = merged_df['arrival_time'].fillna(merged_df['departure.time'].apply(lambda x: x.strftime('%H:%M:%S')))
print(merged_df)

merged_df['arrival_time'] = merged_df['arrival_time'].apply(scheduleDateTime)

merged_df['delay']=merged_df['arrival_time']-merged_df['departure.time']

merged_df['delay'] = merged_df['delay'].apply(lambda x: x.total_seconds() / 60)


print(merged_df)

   stopId            departure.time      id arrival_time  stop_sequence
6   wsrod 2023-03-28 21:45:00+02:00  D-3093     21:45:00            0.0
5   wocho 2023-03-28 21:47:00+02:00  D-3093     21:47:00            1.0
12  wzach 2023-03-28 21:50:00+02:00  D-3093     21:50:00            2.0
15  wreor 2023-03-28 21:52:00+02:00  D-3093     21:52:00            3.0
11  walje 2023-03-28 21:54:00+02:00  D-3093     21:54:00            4.0
1   wrako 2023-03-28 21:56:00+02:00  D-3093     21:56:00            5.0
3   wsalo 2023-03-28 21:58:00+02:00  D-3093     21:58:00            6.0
18  opacz 2023-03-28 22:00:00+02:00  D-3093     22:00:00            7.0
10  micha 2023-03-28 22:03:00+02:00  D-3093     22:03:00            8.0
9   regul 2023-03-28 22:05:00+02:00  D-3093     22:05:00            9.0
2   malic 2023-03-28 22:07:00+02:00  D-3093     22:07:00           10.0
20  twork 2023-03-28 22:10:00+02:00  D-3093     22:10:00           11.0
8   prusz 2023-03-28 22:12:00+02:00  D-3093     22:12:00        